In [2]:
import pandas as pd
from dwave.system import LeapHybridCQMSampler, LeapHybridDQMSampler
from dimod import ConstrainedQuadraticModel, BinaryQuadraticModel, QuadraticModel, DiscreteQuadraticModel

In [8]:
df = pd.read_csv('/workspace/Attempt_optimizer/attempt_dw/d-w/cost_knpack.csv', names=['cost', 'weight'])
capacity = int(0.8 * sum(df['weight']))
print("\nSetting weight capacity to 80% of total: {}".format(str(capacity)))


Setting weight capacity to 80% of total: 89


In [7]:
!cd attempt_dw/d-w/

643.80s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [9]:
def build_knapsack_cqm(costs, weights, max_weight):
    """Construct a CQM for the knapsack problem."""
    num_items = len(costs)
    print("\nBuilding a CQM for {} items.".format(str(num_items)))

    cqm = ConstrainedQuadraticModel()
    obj = BinaryQuadraticModel(vartype='BINARY')
    constraint = QuadraticModel()

    for i in range(num_items):
        # Objective is to maximize the total costs
        obj.add_variable(i)
        obj.set_linear(i, -costs[i])
        # Constraint is to keep the sum of items' weights under or equal capacity
        constraint.add_variable('BINARY', i)
        constraint.set_linear(i, weights[i])

    cqm.set_objective(obj)
    cqm.add_constraint(constraint, sense="<=", rhs=max_weight, label='capacity')

    return cqm

sampler = LeapHybridCQMSampler()

In [20]:
cost, weight = [x[0] for x in df.itertuples(index=False, name=None)], [x[0] for x in df.itertuples(index=False, name=None)]

[35, 85, 30, 50, 70, 80, 55]

In [23]:
cqm = build_knapsack_cqm(cost, weight, capacity)
print("Submitting CQM to solver {}.".format(sampler.solver.name))
sampleset = sampler.sample_cqm(cqm, label='Example - Knapsack')


Building a CQM for 7 items.
Submitting CQM to solver hybrid_constrained_quadratic_model_version1.


In [26]:
feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)
best = feasible_sampleset.first
selected_item = [key for key, val in best.sample.items() if val==1.0]

print(f"\nFound best solution at energy {(best.energy)}")
print("\nSelected item numbers (0-indexed):", selected_item)
print(f"\nSelected item weights: {list(df.weight.loc[selected_item])}, total = {sum(list(df.weight.loc[selected_item]))}")
print(f"\nSelected item costs: {list(df.cost.loc[selected_item])}, total = {sum(list(df.cost.loc[selected_item]))}")


Found best solution at energy -85.0

Selected item numbers (0-indexed): [0, 3]

Selected item weights: [12, 17], total = 29

Selected item costs: [35, 50], total = 85


In [27]:
capacity = 30
items = [(4, 370), (9, 1950), (10, 3500), (21, 6700), (17, 6100), (3, 800), (27, 8300)]
prof = [x[1] for x in items]
weight = [x[0] for x in items]
cqm_1 = build_knapsack_cqm(prof, weight, capacity)
print("Submitting CQM to solver {}.".format(sampler.solver.name))
sampleset = sampler.sample_cqm(cqm_1, label='small - Knapsack')


Building a CQM for 7 items.
Submitting CQM to solver hybrid_constrained_quadratic_model_version1.


In [52]:
sampleset

SampleSet(rec.array([([0., 0., 1., 0., 1., 1., 0.], -10400., 1, [ True],  True),
           ([0., 0., 1., 0., 1., 1., 0.], -10400., 1, [ True],  True),
           ([0., 0., 1., 0., 1., 1., 0.], -10400., 1, [ True],  True),
           ([0., 0., 1., 0., 1., 1., 0.], -10400., 1, [ True],  True),
           ([0., 0., 1., 0., 1., 1., 0.], -10400., 1, [ True],  True),
           ([0., 0., 1., 0., 1., 1., 0.], -10400., 1, [ True],  True),
           ([0., 0., 1., 0., 1., 1., 0.], -10400., 1, [ True],  True),
           ([0., 0., 1., 0., 1., 1., 0.], -10400., 1, [ True],  True),
           ([0., 0., 1., 0., 1., 1., 0.], -10400., 1, [ True],  True),
           ([0., 0., 1., 0., 1., 0., 0.],  -9600., 1, [ True],  True),
           ([0., 0., 1., 0., 1., 1., 0.], -10400., 1, [ True],  True),
           ([0., 0., 1., 0., 1., 1., 0.], -10400., 1, [ True],  True),
           ([0., 0., 1., 0., 1., 1., 0.], -10400., 1, [ True],  True),
           ([0., 0., 1., 0., 1., 1., 0.], -10400., 1, [ True],  Tru

In [28]:
feasible = sampleset.filter(lambda row: row.is_feasible)
best_find = feasible.first
item = [key for key, val in best_find.sample.items() if val==1.0]

In [29]:
print(f'\nFound best solution with energy {(best_find.energy)}'+
      f'\nSelected item numbers (0-indexed): {item}')


Found best solution with energy -10400.0
Selected item numbers (0-indexed): [2, 4, 5]


In [30]:
print(f'Choosen weight: {[weight[i] for i in item]}, total={sum([weight[i] for i in item])}\n'+
      f'And there costs: {[prof[i] for i in item]}, total={sum([prof[i] for i in item])}')

Choosen weight: [10, 17, 3], total=30
And there costs: [3500, 6100, 800], total=10400


### Attempt with anouther Hamiltonian

In [3]:
def build_knapsack_dqm(costs, weights, numbers, capacity):
  """Construct DQM for the knapsack problem
    Args:
        costs (array-like):
            Array of costs associated with the items
        weights (array-like):
            Array of weights associated with the items
        numbers (array-like):
            Array of maximum quantity of each item that is available
        capacity (int):
            Maximum allowable weight
    Returns:
        Discrete quadratic model instance
  """
  # intialize DQM
  dqm = DiscreteQuadraticModel()
  const = max(costs)
  
  # Build the DQM starting by adding variables
  for item in range(len(costs)):
      dqm.add_variable(numbers[item] + 1, label = 'x' + str(item))
  # add y
  dqm.add_variable(capacity, label = 'y')
  # Hamiltonian xi terms
  for k in range(len(costs)):
      dqm.set_linear('x' + str(k), [-costs[k] * i + const * pow(weights[k],2) * pow(i,2) for i in range(numbers[k] + 1)])

  # Hamiltonian xi-xj terms
  for i in range(len(costs)):
        for j in range(i + 1, len(costs)):
            xixj_values = [[xi * xj * 2 * const * weights[i] * weights[j] for xi in range(numbers[i] + 1)] for xj in range(numbers[j] + 1)]
            dqm.set_quadratic('x' + str(i), 'x' + str(j), xixj_values)

  # Hamiltonian y-y term
  dqm.set_linear('y', [const * pow(i,2) for i in range(1,capacity + 1)])
  # Hamiltonian x-y terms
  for i in range(len(costs)):
    xy_values = [[-2 * const * weights[i] * x * y for x in range(numbers[i] + 1)] for y in range(1,capacity + 1)]
    dqm.set_quadratic('x' + str(i), 'y', xy_values)

  return dqm

In [62]:
capacit = 89

In [63]:
df['num'] = [3,2,4,1,4,3,1]
dqm = build_knapsack_dqm(df.cost.to_list(),df.weight.to_list(),df.num.to_list(),capacit)
sampler2 = LeapHybridDQMSampler()
response = sampler2.sample_dqm(dqm, label='Example_2 - Knapsack')

In [51]:
response

SampleSet(rec.array([([ 0,  0,  0,  0,  1,  3,  1, 29], -209890., 1),
           ([ 0,  0,  0,  0,  1,  1,  1, 29], -374630., 1),
           ([ 0,  0,  0,  0,  1,  1,  1, 29], -374630., 1),
           ([ 0,  0,  0,  0,  1,  1,  1, 29], -374630., 1),
           ([ 0,  0,  0,  0,  1,  1,  1, 29], -374630., 1),
           ([ 0,  0,  0,  0,  1,  1,  1, 29], -374630., 1),
           ([ 0,  0,  0,  0,  1,  1,  1, 29], -374630., 1),
           ([ 0,  0,  0,  0,  1,  1,  1, 29], -374630., 1),
           ([ 0,  0,  0,  0,  1,  1,  1, 29], -374630., 1),
           ([ 0,  0,  0,  0,  1,  1,  1, 29], -374630., 1),
           ([ 0,  0,  0,  0,  1,  1,  1, 29], -374630., 1),
           ([ 0,  0,  0,  0,  1,  1,  1, 29], -374630., 1),
           ([ 0,  0,  0,  0,  1,  1,  1, 29], -374630., 1),
           ([ 0,  0,  0,  0,  1,  1,  1, 29], -374630., 1),
           ([ 0,  0,  0,  0,  1,  1,  1, 29], -374630., 1),
           ([ 0,  0,  0,  0,  1,  1,  1, 29], -374630., 1),
           ([ 0,  0,  0,  0,  

In [68]:
sample = response.first.sample
energy = response.first.energy
print(f'Sample: {sample} with energy: {energy}')

Sample: {'x0': 2, 'x1': 2, 'x2': 2, 'x3': 0, 'x4': 2, 'x5': 2, 'x6': 0, 'y': 61} with energy: -1238200.0


In [70]:
response

SampleSet(rec.array([([ 2,  2,  2,  0,  2,  2,  0, 61], -1238200., 1),
           ([ 2,  2,  2,  0,  2,  2,  0, 61], -1238200., 1),
           ([ 2,  2,  2,  0,  2,  2,  0, 61], -1238200., 1),
           ([ 2,  2,  2,  0,  2,  2,  0, 61], -1238200., 1),
           ([ 2,  2,  2,  0,  2,  2,  0, 61], -1238200., 1),
           ([ 2,  2,  2,  0,  2,  2,  0, 61], -1238200., 1),
           ([ 2,  2,  2,  0,  2,  2,  0, 61], -1238200., 1),
           ([ 2,  2,  2,  0,  2,  2,  0, 61], -1238200., 1),
           ([ 2,  2,  2,  0,  2,  2,  0, 61], -1238200., 1),
           ([ 2,  2,  2,  0,  2,  2,  0, 61], -1238200., 1),
           ([ 2,  2,  2,  0,  2,  2,  0, 61], -1238200., 1),
           ([ 2,  2,  2,  0,  2,  2,  0, 61], -1238200., 1),
           ([ 2,  2,  2,  0,  2,  2,  0, 61], -1238200., 1),
           ([ 2,  2,  2,  0,  2,  2,  0, 61], -1238200., 1),
           ([ 2,  2,  2,  0,  2,  2,  0, 61], -1238200., 1),
           ([ 2,  2,  2,  0,  2,  2,  0, 61], -1238200., 1),
           ([ 

In [71]:
selected = {}
for varname, value in sample.items():
    if (value>1) and varname.startswith('x'):
        selected[int(varname[1:])] = value

In [72]:
selected

{0: 2, 1: 2, 2: 2, 4: 2, 5: 2}

In [73]:
selected_item = [key for key, val in selected.items() if val==2.0]

best = feasible_sampleset.first
print(f"\nFound best solution at energy {(energy)}")
print("\nSelected item numbers (0-indexed):", selected_item)
print(f"\nSelected item weights: {list(df.weight.loc[selected_item])}, total = {sum(list(df.weight.loc[selected_item]))}")
print(f"\nSelected item costs: {list(df.cost.loc[selected_item])}, total = {sum(list(df.cost.loc[selected_item]))}")


Found best solution at energy -1238200.0

Selected item numbers (0-indexed): [0, 1, 2, 4, 5]

Selected item weights: [12, 27, 11, 20, 10], total = 80

Selected item costs: [35, 85, 30, 70, 80], total = 300


In [8]:
capacity = 30
items = [(4, 370), (9, 1950), (10, 3500), (21, 6700), (17, 6100), (3, 800), (27, 8300)]
prof = [x[1] for x in items]
weight = [x[0] for x in items]
dqm_1 = build_knapsack_dqm(prof,weight,[5,5,4,4,3,6,1],capacity)
sampler3 = LeapHybridDQMSampler()
response = sampler3.sample_dqm(dqm_1, label='small_1 - Knapsack')
print("Submitting DQM to solver {}.".format(sampler3.solver.name))

Submitting DQM to solver hybrid_discrete_quadratic_model_version1.


In [9]:
response

SampleSet(rec.array([([ 1,  1,  1,  0,  1,  1,  1, 29], -33619420., 1),
           ([ 1,  1,  1,  0,  1,  1,  1, 29], -33619420., 1),
           ([ 1,  1,  1,  0,  1,  1,  1, 29], -33619420., 1),
           ([ 1,  1,  1,  0,  1,  1,  1, 29], -33619420., 1),
           ([ 1,  1,  1,  0,  1,  1,  1, 29], -33619420., 1),
           ([ 1,  1,  1,  0,  1,  1,  1, 29], -33619420., 1),
           ([ 1,  1,  1,  0,  1,  1,  1, 29], -33619420., 1),
           ([ 1,  1,  1,  0,  1,  1,  1, 29], -33619420., 1),
           ([ 1,  1,  1,  0,  1,  1,  1, 29], -33619420., 1),
           ([ 1,  1,  1,  0,  1,  1,  1, 29], -33619420., 1),
           ([ 1,  1,  1,  0,  1,  1,  1, 29], -33619420., 1),
           ([ 1,  1,  1,  0,  1,  1,  1, 29], -33619420., 1),
           ([ 1,  1,  1,  0,  1,  1,  1, 29], -33619420., 1),
           ([ 1,  1,  1,  0,  1,  1,  1, 29], -33619420., 1),
           ([ 1,  1,  1,  0,  1,  1,  1, 29], -33619420., 1),
           ([ 1,  1,  1,  0,  1,  1,  1, 29], -33619420., 1)

In [14]:
sample = response.first.sample
energy = response.first.energy
selected = {}
for varname, value in sample.items():
    if (value>0) and varname.startswith('x'):
        selected[int(varname[1:])] = value

selected_item = [key for key, val in selected.items() if val==1.0]
print(f"\nFound best solution at energy {(energy)}")
print("\nSelected item numbers (0-indexed):", selected_item)
print(f'Choosen weight: {[weight[i] for i in selected_item]}, total={sum([weight[i] for i in selected_item])}\n'+
      f'And there costs: {[prof[i] for i in selected_item]}, total={sum([prof[i] for i in selected_item])}')


Found best solution at energy -33619420.0

Selected item numbers (0-indexed): [0, 1, 2, 4, 5, 6]
Choosen weight: [4, 9, 10, 17, 3, 27], total=70
And there costs: [370, 1950, 3500, 6100, 800, 8300], total=21020


In [13]:
 selected

{0: 1, 1: 1, 2: 1, 4: 1, 5: 1, 6: 1}

In [ ]:
number_of_items = list(selected_items.values())
[selected_weights[i]*number_of_items[i] for i in range(len(selected_weights))]